# **Introducción a los Servicios Fundamentales de AWS e implementación de seguridad**

# Introducción

En este notebook, exploraremos cuatro servicios fundamentales de AWS:

1. **IAM (Identity and Access Management)** 🛡️
2. **AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL** 🌐
3. **Máquinas Virtuales (EC2)** 💻
4. **Contenedores (ECS)** 📦
5. **Computación Serverless (Lambda)** ⚡

Veremos las características de cada uno y cómo se implementan en AWS mediante este ejercicio.

## Diagrama de ejercicio general

![Diagram](assets/diagram.png)

## Explicación Arquitectura en AWS 

En este ejercicio vamos a crear una arquitectura en AWS que utiliza una VPC con subnets públicas y privadas:

- **Subnet pública**: alojará una aplicación web. 🌐
- **Subnet privada**: contendrá una base de datos. 💾

## Componentes clave:

- **Internet Gateway**: permitirá el acceso externo. 🌍
- **NAT Gateway**: permitirá que la base de datos acceda a internet sin ser expuesta. 🔒
- **Security Groups**: protegerán los recursos controlando el tráfico. 🛡️
- **ECR (Elastic Container Registry)**: almacenará imágenes de contenedores utilizadas por la aplicación. 📦
- **Lambda**: se empleará para hacer llamadas al servicio web. ⚡
- **S3**: almacenará los logs de las llamadas Lambda. La Lambda tendrá un rol con acceso exclusivo a un bucket S3. 🗃️
- **Session Manager**: se usará para una conexión segura a las instancias. 🔐
- **Tablas de enrutamiento**: gestionarán el flujo del tráfico dentro de la VPC. 🛣️

Esta arquitectura asegura una comunicación eficiente y segura entre los diferentes componentes.


# IAM (Identity and Access Management)

**IAM** es un servicio web que ayuda a controlar de forma segura el acceso a los recursos de AWS. 🛡️

Características:
- Gestión centralizada de usuarios y accesos 👥
- Autenticación y autorización granular 🔐
- Integración con servicios AWS 🔗
- Soporte para identidades federadas 🌍
- Sin costo adicional 💸


## Conceptos Clave

- **Cuenta Root**: Al crear una cuenta de AWS, se crea una cuenta root por defecto. Esta cuenta tiene acceso completo a todos los recursos y servicios de AWS y **no debe ser compartida**. 🔑
- **Usuarios**: Son entidades que creas en AWS para representar personas o servicios que interactúan con los recursos de AWS. Cada usuario tiene un conjunto único de credenciales. 👤
- **Grupos**: Son colecciones de usuarios. No pueden contener otros grupos, solo usuarios. Los usuarios pueden pertenecer a uno o más grupos. 👥
- **Roles**: Son similares a los usuarios, pero están destinados a ser asumidos por cualquier entidad que necesite acceso temporal a recursos específicos. 🎭
- **Políticas**: Definen permisos y pueden ser adjuntadas a usuarios, grupos o roles. 📜


### Ejemplo: Asignar nuevo usuario grupo con acceso a S3

En este ejercicio, vamos a crear un usuario y un grupo en AWS. El grupo tendrá permisos exclusivamente para enlistar y crear buckets en S3.


In [6]:
from IPython.display import IFrame

pdf_path = 'assets/iam.pdf'

IFrame(pdf_path, width=1000, height=500)


### Ejemplo: Crear un Rol de Administrador y Asumirlo por el Usuario
En este ejercicio, crearemos un rol de tipo Administrador, que tendrá permisos para crear y leer cualquier recurso en la cuenta de AWS. Además, otorgaremos al usuario los permisos necesarios para asumir este rol y ejecutar dichas acciones

In [2]:
from IPython.display import IFrame

pdf_path = 'assets/roleassume.pdf'

IFrame(pdf_path, width=1000, height=500)

### Documentación

Consultar esta información para más detalles:
- [Crear usuarios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_users_create.html)
- [Crear grupos en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_groups_create.html)
- [Crear roles para servicios en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create_for-service.html)
- [Crear roles en IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create.html)
- [Roles de IAM](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles.html)
- [Asumir roles de AWS](https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_use.html)

# Setup Inicial para ejecutar proyecto 🚀

### Importar bibliotecas 🚀

In [2]:
import boto3

### Configurar el cliente de AWS (asegúrate de tener tus credenciales configuradas) 🚀

In [3]:
iam = boto3.client('iam')
ec2 = boto3.client('ec2')
lambda_client = boto3.client('lambda')
ecs = boto3.client('ecs')

# AWS Networking: VPC, Subnets, NAT, Internet Gateway, Route Tables, ACL 🌐

AWS Networking se centra en la configuración y gestión de redes dentro de AWS utilizando varios componentes clave:

- **VPC (Virtual Private Cloud):** Una red virtual aislada dentro de AWS donde se despliegan recursos. 🏠
- **Subnets:** Segmentos de una VPC que permiten organizar y asegurar los recursos de manera granular. 📊
- **NAT (Network Address Translation):** Permite que instancias en subnets privadas accedan a Internet sin exponer sus direcciones IP privadas. 🔄
- **Internet Gateway:** Un componente que permite la comunicación entre instancias en una VPC y la Internet. 🌍
- **Route Tables:** Definen las reglas de enrutamiento para controlar el tráfico dentro de la VPC y hacia fuera. 🛣️
- **ACL (Access Control Lists):** Listas de control de acceso que proporcionan una capa de seguridad adicional a nivel de subred, permitiendo o denegando tráfico. 🔒

Estos componentes juntos permiten diseñar y gestionar redes seguras y eficientes en la nube de AWS.


# Máquinas Virtuales (EC2 en AWS) 💻

Las máquinas virtuales son emulaciones de sistemas informáticos completos. En AWS, se implementan como instancias EC2 (Elastic Compute Cloud).

Características:
- Emula hardware completo 🖥️
- Control total sobre el sistema operativo ⚙️
- Requiere gestión y mantenimiento del sistema operativo 🛠️
- Escalado manual o automatizado, pero no instantáneo 📈
- Facturación por hora/segundo de ejecución ⏲️

## Ejemplo: Instancia EC2 AWS 🚀

In [ ]:
def create_ec2_instance():
    response = ec2.run_instances(
        ImageId='ami-xxxxxxxx',  # AMI ID (ejemplo)
        InstanceType='t2.micro',
        MinCount=1,
        MaxCount=1
    )
    return response['Instances'][0]['InstanceId']

instance_id = create_ec2_instance()
print(f"Instancia EC2 creada con ID: {instance_id}")

# Serverless (AWS Lambda) ⚡

La computación serverless permite ejecutar código sin provisionar ni administrar servidores. AWS Lambda es el servicio serverless de AWS.

Características:
- No hay que gestionar servidores 🛠️
- Escalado automático e instantáneo 📈
- Facturación por milisegundo de ejecución ⏱️
- Ideal para cargas de trabajo intermitentes 🌐
- Limitado en tiempo de ejecución y recursos ⏳

### Ejemplo: Crear una función Lambda

In [ ]:
def create_lambda_function():
    with open('lambda_function.zip', 'rb') as f:
        zipped_code = f.read()
    
    response = lambda_client.create_function(
        FunctionName='MyLambdaFunction',
        Runtime='python3.8',
        Role='arn:aws:iam::XXXXXXXXXXXX:role/lambda-role',  # IAM role ARN
        Handler='lambda_function.handler',
        Code=dict(ZipFile=zipped_code),
        Timeout=30
    )
    return response['FunctionArn']

lambda_arn = create_lambda_function()
print(f"Función Lambda creada con ARN: {lambda_arn}")

## Contenedores (ECS en AWS)

Los contenedores son unidades estándar de software que empaquetan el código y todas sus dependencias.
AWS ECS (Elastic Container Service) es un servicio de orquestación de contenedores.

Características:
- Ligeros y portables
- Aislamiento de aplicaciones
- Rápido despliegue y escalado
- Eficiente uso de recursos
- Ideal para microservicios

### Ejemplo: Crear un cluster ECS

In [ ]:
def create_ecs_cluster(cluster_name):
    response = ecs.create_cluster(clusterName=cluster_name)
    return response['cluster']['clusterArn']

cluster_arn = create_ecs_cluster('MiClusterECS')
print(f"Cluster ECS creado con ARN: {cluster_arn}")

## Comparación de servicios

1. IAM:
   - Gestión centralizada de accesos
   - Crucial para la seguridad en la nube
   - Se integra con todos los servicios AWS
   - Sin costo adicional

2. Máquinas Virtuales (EC2):
   - Mayor control y flexibilidad
   - Requiere más gestión
   - Mejor para aplicaciones que necesitan el sistema operativo completo
   - Costo: Puede ser más alto para cargas de trabajo intermitentes

3. Serverless (Lambda):
   - Sin gestión de servidores
   - Escalado automático
   - Ideal para microservicios y funciones específicas
   - Costo: Muy eficiente para cargas de trabajo variables

4. Contenedores (ECS):
   - Portabilidad y consistencia
   - Menos sobrecarga que las VMs
   - Bueno para microservicios y aplicaciones distribuidas
   - Costo: Equilibrio entre VMs y serverless

La elección depende de los requisitos específicos de tu aplicación, 
la experiencia de tu equipo y las necesidades de gestión y escalabilidad.